## Setup

In [1]:
import os

In [2]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False
print(IN_COLAB)

False


In [3]:
!pip install graph_nets  dm-sonnet tensorflow_probability

You should consider upgrading via the 'C:\Users\david\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
if IN_COLAB and 'drive' not in os.listdir("/content"):
    from google.colab import drive

    drive.mount('/content/drive')

In [5]:
if IN_COLAB:
    os.chdir('/content/drive/MyDrive/bakalarka')

In [6]:
import tensorflow as tf

import time
import sys
import re

from graph_nets import utils_tf


In [7]:
from EncodeProcessDecodeBasic import EncodeProcessDecode
from EncodeProcessDecodeNorm import EncodeProcessDecode as EncodeProcessDecodeNorm
from EncodeProcessDecodeAddNorm import EncodeProcessDecode as EncodeProcessDecodeAddNorm
from EncodeProcessDecodeNoLib import EncodeProcessDecode as EncodeProcessDecodeNoLib
from EncodeProcessDecodeMultinet import EncodeProcessDecode as EncodeProcessDecodeMultinet
from EncodeProcessDecodeNamedMultinet import EncodeProcessDecode as EncodeProcessDecodeNamedMultinet

In [8]:
mod = "named-full"
models = {"basic": {"model": EncodeProcessDecode,
                    "args": {"steps": 15,
                             "learn_features": 2,
                             "node_net_sizes": [128] * 2 + [2],
                             "edge_net_sizes": [128] * 2 + [4]},
                    "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                    size=2, batch=1)]},
          "small_test": {"model": EncodeProcessDecode,
                         "args": {"steps": 5,
                                  "learn_features": 2,
                                  "node_net_sizes": [8] * 2 + [2],
                                  "edge_net_sizes": [8] * 2 + [4]},
                         "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                         size=2, batch=1)]},
          "full": {"model": EncodeProcessDecodeNorm,
                   "args": {"steps": 15,
                            "learn_features": 2,
                            "n_layers": 2,
                            "lat_size": 128,
                            "edge_feat_cnt": 3,
                            "node_feat_cnt": 5},
                   "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                   size=2, batch=1)]},
          "no-norm": {"model": EncodeProcessDecodeNorm,
                      "args": {"steps": 15,
                               "learn_features": 2,
                               "n_layers": 2,
                               "lat_size": 128,
                               "edge_feat_cnt": 3,
                               "node_feat_cnt": 5,
                               "la_norm": False},
                      "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                      size=2, batch=1)]},
          "add": {"model": EncodeProcessDecodeAddNorm,
                  "args": {"steps": 15,
                           "learn_features": 2,
                           "n_layers": 2,
                           "lat_size": 128,
                           "edge_feat_cnt": 3,
                           "node_feat_cnt": 5},
                  "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                  size=2, batch=1)]},
          "no-lib": {"model": EncodeProcessDecodeNoLib,
                     "args": {"steps": 15,
                              "learn_features": 2,
                              "n_layers": 2,
                              "lat_size": 128,
                              "edge_feat_cnt": 3,
                              "node_feat_cnt": 5},
                     "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                     size=2, batch=1)]},
          "multinet": {"model": EncodeProcessDecodeMultinet,
                       "args": {"steps": 15,
                                "learn_features": 2,
                                "n_layers": 2,
                                "lat_size": 128,
                                "edge_feat_cnt": 3,
                                "node_feat_cnt": 5},
                       "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                       size=2, batch=2)]},
          "named": {"model": EncodeProcessDecodeNamedMultinet,
                    "args": {"steps": 15,
                             "learn_features": 2,
                             "n_layers": 2,
                             "lat_size": 128,
                             "edge_feat_cnt": 3,
                             "node_feat_cnt": 5},
                    "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                    size=2, batch=2)]},
          "named18": {"model": EncodeProcessDecodeNamedMultinet,
                      "args": {"steps": 18,
                               "learn_features": 2,
                               "n_layers": 2,
                               "lat_size": 128,
                               "edge_feat_cnt": 3,
                               "node_feat_cnt": 5},
                      "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                      size=2, batch=2)]},
          "named12": {"model": EncodeProcessDecodeNamedMultinet,
                      "args": {"steps": 12,
                               "learn_features": 2,
                               "n_layers": 2,
                               "lat_size": 128,
                               "edge_feat_cnt": 3,
                               "node_feat_cnt": 5},
                      "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                      size=2, batch=2)]},
          "named-full": {"model": EncodeProcessDecodeNamedMultinet,
                         "args": {"steps": 12,
                                  "learn_features": 4,
                                  "n_layers": 2,
                                  "lat_size": 128,
                                  "edge_feat_cnt": 3,
                                  "node_feat_cnt": 7},
                         "params": [dict(noise=0.02, gamma=1.0, field='velocity', history=False,
                                         size=2, batch=2),
                                    dict(noise=0.02, gamma=1.0, field='pressure', history=False,
                                         size=1, batch=2),
                                    dict(noise=0.02, gamma=1.0, field='density', history=False,
                                         size=1, batch=2)]},
          "named12n": {"model": EncodeProcessDecodeNamedMultinet,
                    "args": {"steps": 12,
                             "learn_features": 2,
                             "n_layers": 2,
                             "lat_size": 128,
                             "edge_feat_cnt": 3,
                             "node_feat_cnt": 5},
                    "params": [dict(noise=1.0, gamma=1.0, field='velocity', history=False,
                                    size=2, batch=2)]}}

In [9]:
params = models[mod]["params"]
targets = tuple([x["field"] for x in params])

In [10]:
from common import NodeType

## Data Loading

In [11]:
data_path = 'data/airfoil'
data_train = 'small' if IN_COLAB else 'train'
data_valid = 'small' if IN_COLAB else 'valid'
data_test = 'small' if IN_COLAB else 'test'

In [12]:
from dataset import load_dataset, split_dataset, prepare, add_noises, add_targets, add_batch, triangles_to_edges

In [13]:
ds = load_dataset(data_path, data_train)
ds = add_targets(ds, targets, add_history=params[0]['history'])

In [14]:
ds = split_dataset(ds)
for param in params:
    ds = add_noises(ds, noise_field=param['field'],
                    noise_scale=param['noise'],
                    noise_gamma=param['gamma'])
ds = prepare(ds, 100 if IN_COLAB else 10000)

In [15]:
ds = add_batch(ds, params[0]['batch'])

## Prepare for learning

In [16]:
itr = iter(ds)
d = itr.next()

In [17]:
NodeTypeCnt = tf.unique(tf.reshape(d['node_type'], d['node_type'].shape[:1])).y.shape[0]

In [18]:
@tf.function
def toGraphsTuple(d, targets=('velocity',)):
    send, recive = triangles_to_edges(d['cells'])
    rel_pos = (tf.gather(d['mesh_pos'], send) - tf.gather(d['mesh_pos'], recive))
    nodes_unique = tf.unique_with_counts(tf.reshape(d["node_type"], [-1]))
    one_hot = tf.one_hot(nodes_unique.idx, NodeTypeCnt, dtype=tf.float32)
    dd = {
        "nodes": tf.concat([*[d[x] for x in targets], one_hot], 1),
        # on change update loss function ^
        "senders": send,
        "receivers": recive,
        "edges": tf.concat([
            rel_pos,
            tf.norm(rel_pos, axis=-1, keepdims=True)], 1)
    }
    return utils_tf.data_dicts_to_graphs_tuple([dd])

In [19]:
dd = toGraphsTuple(d, targets)
LINEAR_FEATURES = dd.nodes.shape[1] - NodeTypeCnt

In [20]:
dd

GraphsTuple(nodes=<tf.Tensor: shape=(10466, 7), dtype=float32, numpy=
array([[6.54558029e+01, 8.08514023e+00, 1.04077398e+05, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.33669281e+01, 1.03619909e+01, 1.03129508e+05, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.75267258e+01, 1.11309643e+01, 1.02929469e+05, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.25082756e+02, 7.78504133e+00, 1.01325000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.25070152e+02, 7.76335239e+00, 1.01324992e+05, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.25066811e+02, 7.70044041e+00, 1.01325008e+05, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]], dtype=float32)>, edges=<tf.Tensor: shape=(61796, 3), dtype=float32, numpy=
array([[ 0.00579996, -0.01206488,  0.01338659],
       [-0.00720367, -0.01440071,  0.01610196],
       [-0.00630522,  0.0145334 ,  0.0

In [21]:
aa = itr.next()

## Model and loss function

### Model hyperparameters

In [22]:
from AdamModLr import Adam

lr = 1e-4
# opt = snt.optimizers.Adam(lr)
opt = Adam(lr)
steps = 15
#model = EncodeProcessDecode(steps, LINEAR_FEATURES, [128] * 2, [128] * 2 + [4])
md = models[mod]
model = md["model"](**md["args"])
initial_learning_rate = lr
decay_rate = 0.1
decay_steps = 5e6
learning_increase = 1e-6

## Training

In [23]:
def decayed_learning_rate(step):
    return initial_learning_rate * decay_rate ** (step / decay_steps) + learning_increase

In [24]:
@tf.function
def update_step(data, targets=('velocity')):
    print("Tracing!")
    grp = toGraphsTuple(data, targets)
    with tf.GradientTape() as tape:
        los = model.loss(grp, data, targets=targets)  #change to loss

    gradients = tape.gradient(los, model.trainable_variables)

    opt.apply(gradients, model.trainable_variables)
    return los

In [26]:
update_step(aa, targets);

Tracing!


In [27]:
@tf.function
def loss(grp, data):
    return model.loss(grp, data)


In [28]:
if IN_COLAB:
    logdir = 'logs/tst/'
    writer = tf.summary.create_file_writer(logdir)

    # Bracket the function call with
    # tf.summary.trace_on() and tf.summary.trace_export().
    tf.summary.trace_on(graph=True, profiler=True)
    # Call only one tf.function when tracing.
    z = loss(dd, d)
    with writer.as_default():
        tf.summary.trace_export(
            name="my_func_trace",
            step=0,
            profiler_outdir=logdir)

In [29]:
chck_root = "models/fullnet/"
chck_name = mod

chck = tf.train.Checkpoint(module=model)

latest = tf.train.latest_checkpoint(chck_root)
if latest is not None:
    print("loading", latest)
    chck.restore(latest)

In [30]:
if latest is None and not isinstance(model, EncodeProcessDecode):
    print("Acumulating")
    for i in range(1000):
        data = itr.next()
        grp = toGraphsTuple(data, targets)
        model.loss(grp, data,targets=targets)

Acumulating



KeyboardInterrupt



In [ ]:
if latest is None and not isinstance(model, EncodeProcessDecode):
    print("Acumulating")
    for i in range(1000):
        data = itr.next()
        grp = toGraphsTuple(data)
        model.loss(grp, data)

In [ ]:
dt2 = load_dataset(data_path, data_train)
dt2 = add_targets(dt2, targets, add_history=params[0]['history'])
qq = iter(dt2).next()
qqq = {}
for i, j in qq.items():
    qqq[i] = j[0]
grp_ = toGraphsTuple(qqq, targets)

In [ ]:
tf.reduce_min(qqq['density']), tf.reduce_max(qqq['density']),tf.reduce_min(qqq['velocity'][0]), tf.reduce_max(qqq['velocity'][0]),tf.reduce_min(qqq['pressure']), tf.reduce_max(qqq['pressure'])

In [ ]:
if "m" in locals():
    m.numpy()

## Vizualization

In [ ]:
res = [grp_, ]
loss_mask = tf.logical_or(tf.equal(qqq['node_type'][:, 0], NodeType.NORMAL),
                          tf.equal(qqq['node_type'][:, 0], NodeType.OUTFLOW))
feat = grp_.nodes.shape
loss_mask = tf.reshape(tf.concat([loss_mask for _ in range(grp_.nodes.shape[1])], -1), [-1, grp_.nodes.shape[1]])
for i in range(600):
    grp2_ = model(grp_, False)
    grp_ = grp_.replace(nodes=tf.where(loss_mask, grp2_.nodes, grp_.nodes))
    res.append(grp_)

In [ ]:
@tf.function
def toGraphsTupleOld(d):
    send, recive = triangles_to_edges(d['cells'])
    rel_pos = (tf.gather(d['mesh_pos'], send) - tf.gather(d['mesh_pos'], recive))
    nodes_unique = tf.unique_with_counts(tf.reshape(d["node_type"], [-1]))
    dd = {
        #"nodes": tf.concat([d["velocity"],d["pressure"],d["density"],tf.cast(d["node_type"],tf.float32),d["mesh_pos"]],1),
        "nodes": tf.concat([d['velocity'], tf.one_hot(tf.reshape(d["node_type"], [-1]), NodeTypeCnt, dtype=tf.float32)],
                           1),  # on change update loss function ^
        "senders": send,
        "receivers": recive,
        "edges": tf.concat([
            rel_pos,
            tf.norm(rel_pos, axis=-1, keepdims=True)], 1)
    }
    return utils_tf.data_dicts_to_graphs_tuple([dd])

In [ ]:
# fix mistake in data preparation
if chck_root == "models/new":
    grp_ = toGraphsTupleOld(qqq)

In [ ]:
res = [grp_, ]
loss_mask = tf.logical_or(tf.equal(qqq['node_type'][:, 0], NodeType.NORMAL),
                          tf.equal(qqq['node_type'][:, 0], NodeType.OUTFLOW))
loss_mask = tf.reshape(tf.concat([loss_mask for _ in range(5)], -1), [-1, 5])
for i in range(600):
    grp2_ = model(grp_, False)
    grp_ = grp_.replace(nodes=tf.where(loss_mask, grp2_.nodes, grp_.nodes))
    res.append(grp_)

In [ ]:
grp_.nodes.shape, grp2_.nodes.shape, loss_mask.shape

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import tri as mtri

In [ ]:
res[0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
skip = 5
num_steps = len(res)
num_frames = num_steps // skip
# compute bounds
bounds = []
bb_min, bb_max = tf.reduce_min(qq['velocity'][:, 0]), tf.reduce_max(qq['velocity'][:, 0])


def animate(num):
    global t
    step = (num * skip) % num_steps
    traj = (num * skip) // num_steps
    ax.cla()
    ax.set_xlim(-1, 2)
    ax.set_ylim(-1.5, 1.5)
    ax.set_autoscale_on(False)
    vmin, vmax = bb_min, bb_max
    pos = qqq['mesh_pos']
    faces = qqq['cells']
    velocity = res[step].nodes[..., :2].numpy()
    triang = mtri.Triangulation(pos[:, 0], pos[:, 1], faces)
    t = ax.tripcolor(triang, velocity[:, 0], vmin=vmin, vmax=vmax)
    ax.triplot(triang, 'ko-', ms=0.5, lw=0.3)
    ax.set_title('Trajectory %d Step %d' % (traj, step))
    return fig,


animate(0)
plt.colorbar(t)

anim = FuncAnimation(fig, animate, frames=num_frames, interval=200)
from IPython.display import HTML

HTML(anim.to_html5_video())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
skip = 5
num_steps = len(res)
num_frames = num_steps // skip
# compute bounds
bounds = []

bb_min, bb_max = tf.reduce_min(qq['velocity'][:, 0]), tf.reduce_max(qq['velocity'][:, 0])


def animate(num):
    global t
    step = (num * skip) % num_steps
    traj = (num * skip) // num_steps
    ax.cla()
    ax.set_xlim(-1, 2)
    ax.set_ylim(-1.5, 1.5)
    ax.set_autoscale_on(False)
    vmin, vmax = bb_min, bb_max
    pos = qq['mesh_pos'][0]
    faces = qq['cells'][0]
    velocity = qq['velocity'][step]
    triang = mtri.Triangulation(pos[:, 0], pos[:, 1], faces)
    t = ax.tripcolor(triang, velocity[:, 0], vmin=vmin, vmax=vmax)
    ax.triplot(triang, 'ko-', ms=0.5, lw=0.3)
    ax.set_title('Trajectory %d Step %d' % (traj, step))
    return fig,


animate(0)
plt.colorbar(t)

anim = FuncAnimation(fig, animate, frames=num_frames, interval=200)
from IPython.display import HTML

HTML(anim.to_html5_video())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
skip = 5
num_steps = len(res)
num_frames = num_steps // skip
# compute bounds
bounds = []

bb_min, bb_max = 0, tf.reduce_max(qq['velocity'][:, 0])


def animate(num):
    global t
    step = (num * skip) % num_steps
    traj = (num * skip) // num_steps
    ax.cla()
    ax.set_xlim(-1, 2)
    ax.set_ylim(-1.5, 1.5)
    ax.set_autoscale_on(False)
    vmin, vmax = bb_min, bb_max
    pos = qq['mesh_pos'][0]
    faces = qq['cells'][0]
    velocity = tf.math.abs(qq['velocity'][step] - res[step].nodes[..., :2])
    triang = mtri.Triangulation(pos[:, 0], pos[:, 1], faces)
    t = ax.tripcolor(triang, velocity[:, 0], vmin=vmin, vmax=vmax, cmap='Reds')
    ax.triplot(triang, 'ko-', ms=0.5, lw=0.3)
    ax.set_title('Trajectory %d Step %d' % (traj, step))
    return fig,


animate(0)
plt.colorbar(t)

anim = FuncAnimation(fig, animate, frames=num_frames, interval=200)
from IPython.display import HTML

HTML(anim.to_html5_video())

In [ ]:
bb_min, bb_max

In [ ]:
model._node_norm._std_with_epsilon(), model._node_norm._mean()

In [ ]:
qq['target|velocity'].shape, qq['velocity'].shape

In [ ]:
r = tf.reduce_sum((qq['velocity'] - qq['target|velocity']) ** 2)
r

In [ ]:
tt = [{i: qq[i][x] for i in qq.keys()} for x in range(599)]

In [ ]:
lss = []
for i in tt:
    lss.append(model.loss(toGraphsTuple(i), i))

In [ ]:
lss = [x.numpy() for x in lss]

In [ ]:
plt.plot(lss)

In [ ]:
1e7 / (1000 * 600)

In [ ]:
ee = [x.shape.as_list() for x in model.trainable_variables]
sum([(y[0] if len(y) == 1 else y[0] * y[1]) for y in ee])